In [1]:
import os, sys
sys.path.append(os.getcwd())

import time

import numpy as np
#import tensorflow as tf

import language_helpers
import tflib as lib
import tflib.ops.linear
import tflib.ops.conv1d
import tflib.plot


#print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


2023-01-22 16:36:17.125989: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:

import tensorflow.compat.v1 as tf

tf.disable_v2_behavior() 

In [29]:
# Download Google Billion Word at http://www.statmt.org/lm-benchmark/ and
# fill in the path to the extracted files here!
DATA_DIR = '/Users/alejandrorodriguez/Documents/1-billion-word-language-modeling-benchmark-r13output'
if len(DATA_DIR) == 0:
    raise Exception('Please specify path to data directory in gan_language.py!')

BATCH_SIZE = 64 # Batch size
ITERS = 200000 # How many iterations to train for
SEQ_LEN = 32 # Sequence length in characters
DIM = 512 # Model dimensionality. This is fairly slow and overfits, even on
          # Billion Word. Consider decreasing for smaller datasets.
CRITIC_ITERS = 10 # How many critic iterations per generator iteration. We
                  # use 10 for the results in the paper, but 5 should work fine
                  # as well.
LAMBDA = 10 # Gradient penalty lambda hyperparameter.
MAX_N_EXAMPLES = 10000000 # Max number of data examples to load. If data loading
                          # is too slow or takes too much RAM, you can decrease
                          # this (at the expense of having less training data).
                          

def softmax(logits):
    
    return tf.reshape(
        tf.nn.softmax(
            tf.reshape(logits, [-1, len(charmap)])
        ),
        tf.shape(logits)
    )

def make_noise(shape):
    return tf.random_normal(shape)

def ResBlock(name, inputs):

    print(f"ResBlock input dim: {inputs.shape}")
    output = inputs
    output = tf.nn.relu(output)
    output = lib.ops.conv1d.Conv1D(name+'.1', DIM, DIM, 5, output)
    print(f"ResBlock conv1d 1 out dim: {output.shape}")
    output = tf.nn.relu(output)
    print(f"ResBlock conv1d 2 out dim: {output.shape}")
    output = lib.ops.conv1d.Conv1D(name+'.2', DIM, DIM, 5, output)
    return inputs + (0.3*output)

def Generator(n_samples, prev_outputs=None):
    output = make_noise(shape=[n_samples, 128])
    output = lib.ops.linear.Linear('Generator.Input', 128, SEQ_LEN*DIM, output)
    output = tf.reshape(output, [-1, DIM, SEQ_LEN])
    output = ResBlock('Generator.1', output)
    output = ResBlock('Generator.2', output)
    output = ResBlock('Generator.3', output)
    output = ResBlock('Generator.4', output)
    output = ResBlock('Generator.5', output)

    print(f"ResBlock conv1d in dim: {output.shape}")
    output = lib.ops.conv1d.Conv1D('Generator.Output', DIM, len(charmap), 1, output)
    
    print(f"ResBlock conv1d out dim: {output.shape}")
    output = tf.transpose(output, [0, 2, 1])
    
    print(f"ResBlock transpose out dim: {output.shape}")
    output = softmax(output)
    print(f"ResBlock softmax out dim: {output.shape}")
    
    return output

def Discriminator(inputs):
    print(f"Discriminator input dim: {inputs.shape}")
    output = tf.transpose(inputs, [0,2,1])
    print(f"Discriminator transpose out dim: {output.shape}")
    
    output = lib.ops.conv1d.Conv1D('Discriminator.Input', len(charmap), DIM, 1, output)
    print(f"Discriminator conv1d out dim: {output.shape}")
    
    output = ResBlock('Discriminator.1', output)
    output = ResBlock('Discriminator.2', output)
    output = ResBlock('Discriminator.3', output)
    output = ResBlock('Discriminator.4', output)
    output = ResBlock('Discriminator.5', output)
    print(f"Discriminator ResBlock out dim: {output.shape}")
    
    output = tf.reshape(output, [-1, SEQ_LEN*DIM])
    print(f"Discriminator reshape out dim: {output.shape}")
    
    output = lib.ops.linear.Linear('Discriminator.Output', SEQ_LEN*DIM, 1, output)
    print(f"Discriminator linear out dim: {output.shape}")
    
    return output

In [5]:
lib.print_model_settings(locals().copy())

lines, charmap, inv_charmap = language_helpers.load_dataset(
    max_length=SEQ_LEN,
    max_n_examples=MAX_N_EXAMPLES,
    data_dir=DATA_DIR
)

Uppercase local vars:
	BATCH_SIZE: 64
	CRITIC_ITERS: 10
	DATA_DIR: /Users/alejandrorodriguez/Documents/1-billion-word-language-modeling-benchmark-r13output
	DIM: 512
	ITERS: 200000
	LAMBDA: 10
	MAX_N_EXAMPLES: 10000000
	SEQ_LEN: 32
loading dataset...
('B', 'u', 't', ' ', 'I', ' ', 'w', 'o', 'u', 'l', 'd', ' ', 'p', 'u', 't', ' ', 'm', 'o', 'n', 'e', 'y', ' ', 'o', 'n', ' ', 'a', ' ', 'c', 'o', 'm', 'b', 'i')
('"', ' ', 'I', 't', ' ', 'w', 'a', 's', ' ', 'a', ' ', 's', 'i', 'm', 'u', 'l', 't', 'a', 'n', 'e', 'o', 'u', 's', ' ', 'a', 't', 't', 'a', 'c', 'k', ' ', 'a')
('S', 'o', 'm', 'e', 'o', 'n', 'e', ' ', 'n', 'e', 'e', 'd', 's', ' ', 't', 'o', ' ', 't', 'e', 'l', 'l', ' ', 'T', 'a', 'r', 'a', ' ', 'R', 'e', 'i', 'd', ' ')
('E', 'x', 'p', 'e', 'r', 'i', 'e', 'n', 'c', 'e', 'd', ' ', 'B', 'r', 'i', 't', 'o', 'n', ' ', 'C', 'h', 'a', 'r', 'l', 'i', 'e', ' ', 'W', 'e', 'g', 'e', 'l')
('F', 'r', 'a', 'n', 'k', 'l', 'i', 'n', ' ', "'", 's', ' ', 'S', 'o', 'n', 'g', ' ', 'h', 'a', 's', ' ',

In [6]:
real_inputs_discrete = tf.placeholder(tf.int32, shape=[BATCH_SIZE, SEQ_LEN])
real_inputs = tf.one_hot(real_inputs_discrete, len(charmap))
fake_inputs = Generator(BATCH_SIZE)
fake_inputs_discrete = tf.argmax(fake_inputs, fake_inputs.get_shape().ndims-1)

print(f"real_inputs_discrete dim: {real_inputs_discrete.shape}")
print(f"real_inputs dim: {real_inputs.shape}")
print(f"fake_inputs dim: {fake_inputs.shape}")
print(f"fake_inputs_discrete dim: {fake_inputs_discrete.shape}")


Instructions for updating:
`NCHW` for data_format is deprecated, use `NCW` instead


In [28]:
disc_real = Discriminator(real_inputs)


In [30]:

disc_fake = Discriminator(fake_inputs)

disc_cost = tf.reduce_mean(disc_fake) - tf.reduce_mean(disc_real)
gen_cost = -tf.reduce_mean(disc_fake)

# WGAN lipschitz-penalty
alpha = tf.random_uniform(
    shape=[BATCH_SIZE,1,1], 
    minval=0.,
    maxval=1.
)
differences = fake_inputs - real_inputs
interpolates = real_inputs + (alpha*differences)
interpolates

Discriminator input dim: (64, 32, 297)
Discriminator transpose out dim: (64, 297, 32)
Discriminator conv1d out dim: (64, 512, 32)
ResBlock input dim: (64, 512, 32)
ResBlock conv1d 1 out dim: (64, 512, 32)
ResBlock conv1d 2 out dim: (64, 512, 32)
ResBlock input dim: (64, 512, 32)
ResBlock conv1d 1 out dim: (64, 512, 32)
ResBlock conv1d 2 out dim: (64, 512, 32)
ResBlock input dim: (64, 512, 32)
ResBlock conv1d 1 out dim: (64, 512, 32)
ResBlock conv1d 2 out dim: (64, 512, 32)
ResBlock input dim: (64, 512, 32)
ResBlock conv1d 1 out dim: (64, 512, 32)
ResBlock conv1d 2 out dim: (64, 512, 32)
ResBlock input dim: (64, 512, 32)
ResBlock conv1d 1 out dim: (64, 512, 32)
ResBlock conv1d 2 out dim: (64, 512, 32)
Discriminator ResBlock out dim: (64, 512, 32)
Discriminator reshape out dim: (64, 16384)
Discriminator linear out dim: (64, 1)


<tf.Tensor 'add_57:0' shape=(64, 32, 297) dtype=float32>

In [26]:
Discriminator(interpolates)

<tf.Tensor 'Discriminator.Output_5/BiasAdd:0' shape=(64, 1) dtype=float32>

In [31]:
interpolates

<tf.Tensor 'add_57:0' shape=(64, 32, 297) dtype=float32>

In [32]:
lib.print_model_settings(locals().copy())

Uppercase local vars:
	BATCH_SIZE: 64
	CRITIC_ITERS: 10
	DATA_DIR: /Users/alejandrorodriguez/Documents/1-billion-word-language-modeling-benchmark-r13output
	DIM: 512
	ITERS: 200000
	LAMBDA: 10
	MAX_N_EXAMPLES: 10000000
	SEQ_LEN: 32


In [33]:

disc_cost = tf.reduce_mean(disc_fake) - tf.reduce_mean(disc_real)
gen_cost = -tf.reduce_mean(disc_fake)

# WGAN lipschitz-penalty
alpha = tf.random_uniform(
    shape=[BATCH_SIZE,1,1], 
    minval=0.,
    maxval=1.
)
differences = fake_inputs - real_inputs
interpolates = real_inputs + (alpha*differences)
gradients = tf.gradients(Discriminator(interpolates), [interpolates])[0]
slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), reduction_indices=[1,2]))
gradient_penalty = tf.reduce_mean((slopes-1.)**2)
disc_cost += LAMBDA*gradient_penalty
gen_params = lib.params_with_name('Generator')
disc_params = lib.params_with_name('Discriminator')


Discriminator input dim: (64, 32, 297)
Discriminator transpose out dim: (64, 297, 32)
Discriminator conv1d out dim: (64, 512, 32)
ResBlock input dim: (64, 512, 32)
ResBlock conv1d 1 out dim: (64, 512, 32)
ResBlock conv1d 2 out dim: (64, 512, 32)
ResBlock input dim: (64, 512, 32)
ResBlock conv1d 1 out dim: (64, 512, 32)
ResBlock conv1d 2 out dim: (64, 512, 32)
ResBlock input dim: (64, 512, 32)
ResBlock conv1d 1 out dim: (64, 512, 32)
ResBlock conv1d 2 out dim: (64, 512, 32)
ResBlock input dim: (64, 512, 32)
ResBlock conv1d 1 out dim: (64, 512, 32)
ResBlock conv1d 2 out dim: (64, 512, 32)
ResBlock input dim: (64, 512, 32)
ResBlock conv1d 1 out dim: (64, 512, 32)
ResBlock conv1d 2 out dim: (64, 512, 32)
Discriminator ResBlock out dim: (64, 512, 32)
Discriminator reshape out dim: (64, 16384)
Discriminator linear out dim: (64, 1)


In [35]:

gen_train_op = tf.train.AdamOptimizer(learning_rate=1e-4, beta1=0.5, beta2=0.9).minimize(gen_cost, var_list=gen_params)
disc_train_op = tf.train.AdamOptimizer(learning_rate=1e-4, beta1=0.5, beta2=0.9).minimize(disc_cost, var_list=disc_params)

# Dataset iterator
def inf_train_gen():
    while True:
        np.random.shuffle(lines)
        #for i in xrange(0, len(lines)-BATCH_SIZE+1, BATCH_SIZE):
        for i in range(0, len(lines)-BATCH_SIZE+1, BATCH_SIZE):
            yield np.array(
                [[charmap[c] for c in l] for l in lines[i:i+BATCH_SIZE]], 
                dtype='int32'
            )

# During training we monitor JS divergence between the true & generated ngram
# distributions for n=1,2,3,4. To get an idea of the optimal values, we
# evaluate these statistics on a held-out set first.
#true_char_ngram_lms = [language_helpers.NgramLanguageModel(i+1, lines[10*BATCH_SIZE:], tokenize=False) for i in xrange(4)]
#validation_char_ngram_lms = [language_helpers.NgramLanguageModel(i+1, lines[:10*BATCH_SIZE], tokenize=False) for i in xrange(4)]
true_char_ngram_lms = [language_helpers.NgramLanguageModel(i+1, lines[10*BATCH_SIZE:], tokenize=False) for i in range(4)]
validation_char_ngram_lms = [language_helpers.NgramLanguageModel(i+1, lines[:10*BATCH_SIZE], tokenize=False) for i in range(4)]
#for i in xrange(4):
for i in range(4):
    print("validation set JSD for n={}: {}".format(i+1, true_char_ngram_lms[i].js_with(validation_char_ngram_lms[i])))
#true_char_ngram_lms = [language_helpers.NgramLanguageModel(i+1, lines, tokenize=False) for i in xrange(4)]
true_char_ngram_lms = [language_helpers.NgramLanguageModel(i+1, lines, tokenize=False) for i in range(4)]

with tf.Session() as session:

    session.run(tf.initialize_all_variables())

    def generate_samples():
        samples = session.run(fake_inputs)
        samples = np.argmax(samples, axis=2)
        decoded_samples = []
        #for i in xrange(len(samples)):
        for i in range(len(samples)):
            decoded = []
            #for j in xrange(len(samples[i])):
            for j in range(len(samples[i])):
                decoded.append(inv_charmap[samples[i][j]])
            decoded_samples.append(tuple(decoded))
        return decoded_samples

    gen = inf_train_gen()

    #for iteration in xrange(ITERS):
    for iteration in range(ITERS):
        start_time = time.time()

        # Train generator
        if iteration > 0:
            _ = session.run(gen_train_op)

        # Train critic
        #for i in xrange(CRITIC_ITERS):
        for i in range(CRITIC_ITERS):
            _data = gen.next()
            _disc_cost, _ = session.run(
                [disc_cost, disc_train_op],
                feed_dict={real_inputs_discrete:_data}
            )

        lib.plot.plot('time', time.time() - start_time)
        lib.plot.plot('train disc cost', _disc_cost)

        if iteration % 100 == 99:
            samples = []
            #for i in xrange(10):
            for i in range(10):
                samples.extend(generate_samples())

            #for i in xrange(4):
            for i in range(4):
                lm = language_helpers.NgramLanguageModel(i+1, samples, tokenize=False)
                lib.plot.plot('js{}'.format(i+1), lm.js_with(true_char_ngram_lms[i]))

            with open('samples_{}.txt'.format(iteration), 'w') as f:
                for s in samples:
                    s = "".join(s)
                    f.write(s + "\n")

        if iteration % 100 == 99:
            lib.plot.flush()
        
        lib.plot.tick()


validation set JSD for n=1: 0.0009669324514356473
validation set JSD for n=2: 0.01567580406923486
validation set JSD for n=3: 0.0885953191307561
validation set JSD for n=4: 0.2454666793886078
Instructions for updating:
Use `tf.global_variables_initializer` instead.


2023-01-22 19:08:00.612693: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-22 19:08:00.613051: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-01-22 19:08:03.032571: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


AttributeError: 'generator' object has no attribute 'next'